In [17]:
import math
import cv2
import os
import geopandas as gpd
import dask_geopandas
import numpy as np
import multiprocessing
from pyproj import Transformer
from shapely.geometry import Polygon, box
from tqdm.notebook import trange, tqdm
from tqdm.dask import TqdmCallback
from joblib import Parallel, delayed
from dask import compute, delayed
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, visualize
from dask.distributed import performance_report

transformer = Transformer.from_crs(3395, 4326)
invtransformer = Transformer.from_crs(4326,3395)

In [18]:
def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = ((lon_deg + 180.0) / 360.0 * n)
    ytile = ((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def digital_elevation(gdf, cellsize):
    bounds = gdf.total_bounds
    bb0 = math.ceil(max(bounds[2], bounds[0]))
    bb1 = math.floor(min(bounds[2], bounds[0]))
    bb2 = math.ceil(max(bounds[3], bounds[1]))
    bb3 = math.floor(min(bounds[3], bounds[1]))
    
    width = bb0 - bb1
    height = bb2 - bb3
    
    grid = np.zeros((math.ceil(height/cellsize), math.ceil(width/cellsize)))
    heightColumn = gdf.columns.get_loc("height")
    geomColumn = gdf.columns.get_loc("geometry")
    for x in range(bb1,bb0,cellsize):
        for y in range(bb3,bb2,cellsize):
            x0 = x
            x1 = x+cellsize
            y0 = y
            y1 = y+cellsize
            bbox = box(x0,y0,x1,y1)
            intersected = gdf.geometry.sindex.intersection(bbox.bounds)
            
            for i in intersected:
                buildingHeight = gdf.iat[i,heightColumn] # height -- iat is faster than iloc
                buildingGeometry = gdf.iat[i,geomColumn] # geometry
                val = (bbox.intersection(buildingGeometry).area / (bbox.area)) * buildingHeight
                i, j = int((x-bb1)/cellsize), int((y-bb3)/cellsize)
                grid[j][i] += val.sum()
    grid = np.flipud(grid)
    return grid

def create_image(filtered, i, j, zoom, max_height, outputfolder):
    dem = digital_elevation(filtered,10)
    dem_bounds = filtered.total_bounds
    dem_bb0 = transformer.transform(dem_bounds[0],dem_bounds[1])
    dem_bb1 = transformer.transform(dem_bounds[2],dem_bounds[3])
    dem_bb0 = deg2num(dem_bb0[0],dem_bb0[1],zoom)
    dem_bb1 = deg2num(dem_bb1[0],dem_bb1[1],zoom)
    dsize = (math.ceil(256*abs(dem_bb1[0]-dem_bb0[0])),math.ceil(256*abs(dem_bb1[1]-dem_bb0[1])))
    dem = cv2.resize(dem, dsize=dsize)

    startx = 256-256*(dem_bb0[0]%1)
    endx = (-256*(dem_bb1[0]%1))
    starty = 256-256*(dem_bb1[1]%1)
    endy = (-256*(dem_bb0[1]%1))

    values = dem[math.ceil(starty):math.floor(endy),math.ceil(startx):math.floor(endx)]
    if values.shape[0] != 0 and values.shape[1] != 0:

        values = cv2.resize(values, dsize=(256,256))
        filename = '%s/%d/%d/%d.png'%(outputfolder,zoom,i,j)
        directory = os.path.dirname(filename)
        if not os.path.exists(directory):
            os.makedirs(directory)

        success = cv2.imwrite(filename, 255.0 * (values / max_height))
        if not success:
            raise Exception("Could not write image")
#             print('done')
#             aux = mpimg.imread(filename)
#             print(aux.max()*max_height, values.max())

def run(gdf, zoom, max_height, outputfolder):
    bounds = gdf.total_bounds
    lat0,lng0 = transformer.transform(bounds[0],bounds[1])
    lat1,lng1 = transformer.transform(bounds[2],bounds[3])
    coord0 = deg2num(lat0,lng0,zoom)
    coord1 = deg2num(lat1,lng1,zoom)
    bottomleft = [min(coord0[0],coord1[0]),min(coord0[1],coord1[1])]
    topright = [max(coord0[0],coord1[0]),max(coord0[1],coord1[1])]

    width = math.ceil(topright[0])-math.floor(bottomleft[0])
    height = math.ceil(topright[1])-math.floor(bottomleft[1])

    for i in range(math.floor(bottomleft[0]),math.ceil(topright[0])):
        for j in range(math.floor(bottomleft[1]),math.ceil(topright[1])):
            
            bb0 = num2deg(i,j,zoom)
            bb1 = num2deg(i+1,j+1,zoom)
            bb0 = invtransformer.transform(bb0[0],bb0[1])
            bb1 = invtransformer.transform(bb1[0],bb1[1])
            filtered = gdf.cx[bb0[0]:bb1[0],bb0[1]:bb1[1]]
            
            if len(filtered) > 0:
                create_image(filtered, i, j, zoom, max_height, outputfolder)

In [13]:
cities = ['nyc', 'dc', 'la', 'chi', 'bos']
zooms = range(14,18)
max_height = 550

delayed_results = []
for city in cities:
    gdf = gpd.read_feather('data/osm/%s.feather'%city)
    gdf = delayed(gdf)
    for zoom in zooms:   
        delayed_results.append(delayed(run)(gdf, zoom, max_height, 'data/heights/%s/'%city))
compute(*delayed_results, scheduler='processes', num_workers=8)

In [20]:
# from dask.distributed import Client
# client = Client()

In [21]:
# with TqdmCallback(desc="compute"):
#     compute(*delayed_results, scheduler='processes')
# with ProgressBar():
#     compute(*delayed_results, scheduler='processes', n_workers=8)
# with Profiler() as prof, ResourceProfiler() as rprof, CacheProfiler() as cprof:
#         visualize([prof, rprof, cprof])
#         compute(*delayed_results, scheduler='processes', n_workers=8)
# with performance_report(filename="dask-report.html"):
#     compute(*delayed_results, scheduler='processes', num_workers=8)

C:\Miniconda3\envs\geopandas\lib\site-packages\dask\base.py:1283: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# cities = ['nyc', 'dc', 'la', 'chi', 'bos']
# zooms = range(14,18)
# max_height = 550

# delayed_results = []
# for city in cities:
#     gdf = gpd.read_feather('data/osm/%s.feather'%city)
#     for zoom in zooms:   
#         run(gdf, zoom, max_height, 'data/heights/%s/'%city)

In [ ]:
# %lprun -f digital_elevation run()